# PageRank, Markov Chains and Numerical Linear Algebra

--------------------------------
* By: Michail Vazaios (Μιχαήλ Βαζαίος - p3170013)
* E-mail: mich.vaz.99@gmail.com or p3170013@aueb.gr
* Course: "Arithmetic methods in Statistics" (Αριθμητικές μέθοδοι στην Στατιστική)
--------------------------------
#### Outline of project:

1. **Introduction**
2. **Datasets**
    1. Followers on Github
    2. Philosopher Network
3. **Mathematics of PageRank**
4. **Implementation**
5. **Experiments**
6. **Comments on results & final thoughts**
7. **Sources**

## 1. Introduction
--------------------------------

PageRank is an Algorithm for measuring the importance of nodes in directed graphs (although it can be easily extended to work for undirected graphs as well). It is used by Google Search to rank webpages in their search engine results. Of course it is not the only algorithm used by Google for the task of ordering search results, as it is combined with other information retrieval techniques to achieve better results. However, it was the first method used by the company and it remains their most famous algorithm. 

This project aims to implement a simple version of PageRank and test it to measure the importance of nodes in some experimental graphs by comparing the PageRank scores of each node to other metrics of node importance such as In-Degree Centrality, Closeness Centrality, Betweenness Centrality and Eigenvector Centrality.

Note: Throughout this project we use the terms of each of the pairs graph-network, vertex-node, edge-link interchangeably. Generally the words "graph, vertex, edge" are mostly used by mathematicians or when we focus on Graph Theory and the words "network, node, link" are used more in Social Network Analysis concepts. Since this isn't a course in Graph Theory or Social Network Analysis, there is no problem using the one term instead of the other.

## 2. Datasets
--------------------------------

In this section we describe the datasets used and how they were obtained. Since in real life we don't get to work with perfect data gently handled to us, we also chose to try working not only with toy datasets but to also create our own. For the dataset we create ourselves, we state explicitly how it was created (and present the code to create it) and for the one found online, we state how we aquired it and include it in the data folder.

### 2.A Followers on Github

The nodes in this graph will be Github profiles and the edges will be "follower" relationships. An edge from user $A$ to user $B$ means that $A$ follows $B$. To create the graph we start with a Github account (in this case mine) and find all of its neighbours, that is users this account follows or users that follow this account. Then we do the same for the neighbours of the first node and continue to work this way for $N=3$ iterations. This gives us:

- 17 nodes in the first step (followers or following "michalisvaz")
- 115 nodes in the second step (followers or following some of the previous 17 nodes but not the initial node)
- 2557 nodes in the third step (working the same way)

However for the 2557 nodes of the last step we haven't kept track of their neighbors (followers and following). So for these nodes we add as neighbors the (followers and following) nodes that are already in our graph. Of course there are millions of other users on Github, but if we didn't stop at some point, the graph would be huge to run PageRank on it.

Bellow is the code used for this task. Note that if you run this code at another time in the future, you may get different numbers since users may follow or unfollow other users.

In [70]:
from bs4 import BeautifulSoup
import requests, re, argparse, sys, time

# gets links to other accounts from a page
def get_links_to(soup):
    nodes_to = set(soup.find_all("span", {"class": "Link--secondary pl-1"})) | set(soup.find_all("span", {"class": "Link--secondary"}))
    to_return = set()
    for x in nodes_to:
        y = str(x).split(">", 1)[1].split("<", 1)[0]
        new_nodes.add(y)
        to_return.add(y)
    return to_return

N = 3
nodes = {"michalisvaz"}
current_nodes = {"michalisvaz"}
default_url = "https://github.com/"
tab_followers = "?tab=followers"
tab_following = "?tab=following"
graph = dict()
print("-----------------------------")
for i in range(N):
    start = time.time()
    new_nodes = set()
    print("Number of nodes to add their neighbours: ", len(current_nodes))
    for node in current_nodes:
        followers_url = default_url + node + tab_followers
        r = requests.get(followers_url)
        html = r.content
        soup = BeautifulSoup(html, 'html.parser')
        temp1 = get_links_to(soup)
        following_url = default_url + node + tab_following
        r = requests.get(following_url)
        html = r.content
        soup = BeautifulSoup(html, 'html.parser')
        temp2 = get_links_to(soup)
        graph[node] = (temp1, temp2)
    new_nodes = new_nodes - nodes
    nodes = nodes | new_nodes
    current_nodes = new_nodes
    end = time.time()
    print("Time it took: ", end - start, " seconds")
    print("-----------------------------")

# gets links to already existing nodes of the graph
def get_links_to_existing_nodes(soup):
    nodes_to = set(soup.find_all("span", {"class": "Link--secondary pl-1"})) | set(soup.find_all("span", {"class": "Link--secondary"}))
    to_return = set()
    for x in nodes_to:
        y = str(x).split(">", 1)[1].split("<", 1)[0]
        if y in nodes:
            to_return.add(y)
    return to_return

start = time.time()
print("Adding links to existing nodes for the last ", len(current_nodes), " nodes")
for node in current_nodes:
    followers_url = default_url + node + tab_followers
    r = requests.get(followers_url)
    html = r.content
    soup = BeautifulSoup(html, 'html.parser')
    temp1 = get_links_to_existing_nodes(soup)
    following_url = default_url + node + tab_following
    r = requests.get(following_url)
    html = r.content
    soup = BeautifulSoup(html, 'html.parser')
    temp2 = get_links_to_existing_nodes(soup)
    graph[node] = (temp1, temp2)
end = time.time()
print("Time it took: ", end - start, " seconds")
print("-----------------------------")

-----------------------------
Number of nodes to add their neighbours:  1
Time it took:  1.3589646816253662  seconds
-----------------------------
Number of nodes to add their neighbours:  17
Time it took:  21.387369871139526  seconds
-----------------------------
Number of nodes to add their neighbours:  115
Time it took:  187.46242809295654  seconds
-----------------------------
Adding links to existing nodes for the last  2557  nodes
Time it took:  3735.7351458072662  seconds
-----------------------------


Afterwards we match each node name to an id and save the graph to an edgelist of ids. We also create a file to keep the matchings from id to node name and from node name to id stored. We don't need two separate files for the matching since the matching is 1-1. We also don't need to keep track of both followers and following relationships since for each edge "$B$ has $A$ as a follower" there exists an edge "$A$ follows $B$".

In [71]:
cnt = 1
names2ids = dict()
for node in nodes:
    names2ids[node] = cnt
    cnt += 1

edges = set()
for head in graph:
    for tail in graph[head][1]:
        edge = (names2ids[head], names2ids[tail])
        edges.add(edge)
        
with open("data/github_users/id_edgelist.txt", "w", encoding="utf-8") as f:
    for edge in edges:
        f.write(str(edge[0]) + "," + str(edge[1]) + "\n")
with open("data/github_users/matchings.txt", "w", encoding="utf-8") as f:
    for name in names2ids:
        f.write(name + "," + str(names2ids[name]) + "\n")

And below we can see the number of edges and vertices in the graph:

In [72]:
print("Number of vertices: ", len(nodes))
print("Number of edges: ", len(edges))

Number of vertices:  2690
Number of edges:  8402


### 2.B Philosopher Network

This dataset is a graph where the vertices are philosophers and an edge from philosopher $h$ to philosopher $t$ means that $h$'s work infuenced $t$'s work. This graph was handled to us in an edgelist form for a project for the course "Algorithms" (of the Department of Informatics), and can be found under the folder `data/philosophers` with the name `philosophy_edgelist.txt`. One file with the graph as a node-id edgelist, and one with the names-ids matching were also added to this folder (working the same way as with the previous dataset).

Below is the code for this task:

In [164]:
import pandas as pd

original = pd.read_csv("data/philosophers/philosophy_edgelist.txt", 
                       sep="\t", header=None, names=["source name", "target name"])
nodes = set(original["source name"]) | set(original["target name"])

cnt = 1
names2ids = dict()
for node in nodes:
    names2ids[node] = cnt
    cnt += 1
    
original["source id"] = original["source name"].map(names2ids)
original["target id"] = original["target name"].map(names2ids)

original[["source id", "target id"]].to_csv("data/philosophers/id_edgelist.txt", header=False, index=False)
with open("data/philosophers/matchings.txt", "w", encoding="utf-8") as f:
    for name in names2ids:
        f.write(name + "," + str(names2ids[name]) + "\n")

We can also see the number of vertices and edges for the graph:

In [166]:
print("Number of vertices: ", len(nodes))
print("Number of edges: ", original.shape[0])

Number of vertices:  1231
Number of edges:  7303


## Mathematics of PageRank
--------------------------------

The main idea of PageRank is to calculate a score for each page which represents the probability that a person randomly clicking on links will arrive at the certain page. This probability is the PageRank score of the page. If we are talking about graphs in general and not necessarily web pages, then the nodes of the graph are the equivalent of the pages and the edges of the graph are the equivalent of hyperlinks to other pages. We assume that every link in a page is clicked with equal probability (or equivallently for graphs that a user walking randomly across the graph, always chooses a neighbor of their current location(node) uniformly). Based on this the PageRank scores $PR(u)$ for every node $u$ are:

$$PR(u) = \sum_{v \in B(u)} \frac{PR(v)}{L(v)}$$

, where $B(u)$ is the set of nodes $v$ for which there exists an edge $(v, u)$ and $L(v)$ is the out-degree of a node.

In practice we assume that a user who randomly clicks on links will eventually stop clicking. We denote the probability that a user **continues** clicking as $d$ (damping factor) and the PageRank scores become:

$$PR(u) = \frac{1-d}{N} + d\sum_{v \in B(u)} \frac{PR(v)}{L(v)}$$

, where $N$ is the number of nodes in the graph. The damping has been empirically calculated to be around $0.85$ for web pages.

Here we should note that for sinks, which are the pages with no outbound edges, we assume that they have a link to every other page. This is done because otherwise the non-sink pages would be at a huge disadvantage.

### Computation of Pagerank Scores

The PageRank scores can be calculated either iteratively or algebraically. Below we present the two methods.

#### Iterative method

At the start of the iterative method, that is at $t=0$, we assume a uniform probability distribution on the nodes of the graph. So we have:

$$PR(u_i; 0) = \frac{1}{N}$$

, where $N$ is the number of nodes and $PR(u_i; 0)$ is the PageRank score of node $u_i$ at time $t=0$.

At each step of the iteration the PageRank scores are updated by the following formula (based on the rule written above):

$$PR(u_i; t+1) = \frac{1-d}{N} + d \sum_{u_j \in B(u_i)} \frac{PR(u_j; t)}{L(u_j)}$$

where $L(u_j)$ is the number of outbound links from node $u_j$ and $d$ is the damping factor.

The formula can be written in matrix notation as:

$$R(t+1) = dMR(t) + \frac{1-d}{N}\textbf{1} \quad \quad \quad (1)$$

, where $R_i(t) = PR(u_i; t)$, $\textbf{1}$ is the column vector of length $N$ containing only ones and $M$ is defined as:

$$
M = 
\begin{cases}
1/L(u_j),  & \text{if $j$ links to $i$} \\
0, & \text{otherwise}
\end{cases}
$$

The computations stops when the method has converged, that is when:

$$|R(t+1) - R(t)| < \epsilon$$

for some small $\epsilon$.

#### Algebraic method

For $t \rightarrow \infty$ (when the iterative method has converged) the equation $(1)$ becomes:

$$R = dMR + \frac{1-d}{N}\textbf{1}$$

We can solve this system and get:

$$R = (\textbf{I} - dM)^{-1} \frac{1-d}{N} \textbf{1}$$

, where $\textbf{I}$ is the identity matrix. It can be proven that for $d \in (0, 1)$ the solution exists and is unique.


## Implementation
--------------------------------

Below there is an implementation of both the Iterative and the Algebraic method for calculating PageRank. For the algebraic method we check that $0 < d < 1$ so that we don't have problems with existance and uniqueness of solution of the system. For the Iterative Method we pass as an argument to the function a number `epsilon` which is the treshold for the value $|R(t+1) - R(t)|$ to be smaller than it, in order for the method to stop. We also pass the maximum number of iterations to perform. This means that if we have performed more than `max_iter` iterations we stop regardless of having achieved convergence or not. Both functions assume that the sum of each column of the adjacency matrix sums to one.

After the implementation we test our algorithm with a random $1000 \times 1000$ adjacency matrix. After running both versions we print the Pearson's correlation coefficient between the results of the two algorithms. If our implementations are correct (or if they have the same mistake) the coefficient must be very close to $1$. And we can see that indeed it is.

In [2]:
import numpy as np

# The algebraic implementation of PageRank
def PageRank_algebraic(M, d=0.85):
    if d<=0 or d>=1:
        print("d must be greater than 0 and less than 1")
        return None
    if M.shape[0] != M.shape[1] or len(M.shape)!=2:
        print("M must be a square matrix")
        return None
    N = M.shape[0]
    ones = np.ones((N, 1))
    I = np.identity(N)
    return np.matmul(np.linalg.inv(I - d*M) * ((1-d)/N), ones)

# The iterative implementation of PageRank
def PageRank_iterative(M, d=0.85, epsilon=1e-5, max_iter=100):
    if M.shape[0] != M.shape[1] or len(M.shape)!=2:
        print("M must be a square matrix")
        return None
    N = M.shape[0]
    ones = np.ones((N, 1))
    R_current = ones/N
    for i in range(max_iter):
        R_prev = R_current
        R_current = np.matmul(d*M, R_prev) + ((1-d)/N)*ones
        if np.linalg.norm(R_current-R_prev) < epsilon:
            print("Converged after ", i+1, " iterations")
            return R_current
    print("Didn't converge. Stopped after ", max_iter, " iterations")
    return R_current
        

# Regularizes a square matrix so that each column sums to 1. 
# Usually when we say that a matrix is stochastic we mean that 
# its rows sum to 1. Here we need the columns to sum to 1
def make_stochastic(M):
    dimension = M.shape[0]
    if M.shape[0] != M.shape[1] or len(M.shape)!=2:
        print("M must be a square matrix")
        return None
    for c in range(dimension):
        if M[:, c].sum()==0:
            M[:, c] = np.ones(dimension)/dimension
    M = M / M.sum(axis=0)
    return M

dimension = 1000
d = 0.85
M = np.random.randint(2, size=(dimension, dimension))
M = make_stochastic(M)
print("Running algebraic version of PageRank...")
results_alg = PageRank_algebraic(M, d)
# print("Results: ")
# print(results_alg)
print("Running iterative version of PageRank...")
results_iter = PageRank_iterative(M, d)
# print("Results: ")
# print(results_iter)
print("Correlation coefficient between results with the two methods: ")
print(np.corrcoef(results_alg.flatten(), results_iter.flatten())[0][1])

Running algebraic version of PageRank...
Running iterative version of PageRank...
Converged after  3  iterations
Correlation coefficient between results with the two methods: 
0.9999999998045176


## Experiments
--------------------------------

In this section we'll test our implementation of PageRank in the two datasets specified above. For each dataset we'll present the top $k=10$ nodes (in the first case nodes are Github users and in the second case, philosophers). We will also check to what extent the results of PageRank correlate with other measures of node importance in a graph. To calculate these other metrics we'll use `networkX`, a Python library for the creation, manipulation, and study of complex networks. The metrics we'll compare PageRank with, are In-Degree Centrality, Closeness Centrality, Betweenness Centrality and Eigenvector Centrality.

### Followers on Github

Let's start with the "Followers on Github" graph:

In [3]:
import pandas as pd
import numpy as np

edgelist_df = pd.read_csv("data/github_users/id_edgelist.txt", 
                       sep=",", header=None, names=["source id", "target id"])
edgelist = edgelist_df.to_numpy()
adj_matrix = np.zeros((edgelist.max(), edgelist.max()))
for e in edgelist:
    adj_matrix[e[1]-1][e[0]-1] = 1
    
d = 0.85
adj_matrix = make_stochastic(adj_matrix)
print("Running algebraic version of PageRank...")
results_alg = PageRank_algebraic(adj_matrix, d)
# print("Results: ")
# print(results_alg)
print("Running iterative version of PageRank...")
results_iter = PageRank_iterative(adj_matrix, d)
# print("Results: ")
# print(results_iter)
print("Correlation coefficient between results with the two methods: ")
print(np.corrcoef(results_alg.flatten(), results_iter.flatten())[0][1])

Running algebraic version of PageRank...
Running iterative version of PageRank...
Converged after  30  iterations
Correlation coefficient between results with the two methods: 
0.9999998681753948


We can now print the top nodes of our graph based in PageRank scores (we'll use the results from the algebraic method):

In [4]:
nodes_df = pd.read_csv("data/github_users/matchings.txt", 
                       sep=",", header=None, names=["username", "id"])
nodes_df["PageRank"] = results_alg.flatten()
nodes_df.sort_values(by="PageRank", ascending=False, inplace=True)
nodes_df.head(10)

,username,id,PageRank
1217,JakeWharton,1218,0.016931
1914,torvalds,1915,0.014418
2667,dspinellis,2668,0.009126
1105,nat,1106,0.008671
1737,buckyroberts,1738,0.006919
529,chrisbanes,530,0.006801
2595,gaearon,2596,0.006373
819,mitchtabian,820,0.005881
1624,JideGuru,1625,0.005788
1674,yyx990803,1675,0.005649


We see that among the top results are some of the most followed users of Github and also that professor Diomidis Spinellis appears on the top nodes, since he is likely to be followed by nodes close to my profile (probably other AUEB students). Of course, him having a popular Github account with many contributions, plays a vital role as well.

Now we'll check how the PageRank scores for this graph compare with other metrics about the importance of a node. We'll use `networkX` to calculate these metrics and then we'll see the correlation coefficients between PageRank and each of the other metrics.

In [5]:
import networkx as nx

number_of_nodes = edgelist.max()
github_graph = nx.DiGraph()
github_graph.add_edges_from(edgelist)
flat_results = results_alg.flatten()
print("Correlation between PageRank and:")
# In degree
in_degree_dict = nx.in_degree_centrality(github_graph)
in_degree_centrality = np.zeros(number_of_nodes)
for x in in_degree_dict:
    in_degree_centrality[x-1] = in_degree_dict[x]
print("In-degree Centrality : ", np.corrcoef(flat_results, in_degree_centrality)[0][1])
# Closeness Centrality
closeness_dict = nx.closeness_centrality(github_graph)
closeness_centrality = np.zeros(number_of_nodes)
for x in closeness_dict:
    closeness_centrality[x-1] = closeness_dict[x]
print("Closeness Centrality : ", np.corrcoef(flat_results, closeness_centrality)[0][1])
# Betweenness Centrality
betweenness_dict = nx.betweenness_centrality(github_graph)
betweenness_centrality = np.zeros(number_of_nodes)
for x in betweenness_dict:
    betweenness_centrality[x-1] = betweenness_dict[x]
print("Betweenness Centrality : ", np.corrcoef(flat_results, betweenness_centrality)[0][1])
# Eigenvector Centrality
eigenvector_dict = nx.eigenvector_centrality(github_graph)
eigenvector_centrality = np.zeros(number_of_nodes)
for x in eigenvector_dict:
    eigenvector_centrality[x-1] = eigenvector_dict[x]
print("Eigenvector Centrality : ", np.corrcoef(flat_results, eigenvector_centrality)[0][1])

Correlation between PageRank and:
In-degree Centrality :  0.9221225738999792
Closeness Centrality :  0.46330017420505026
Betweenness Centrality :  0.30749575416600966
Eigenvector Centrality :  0.259704009381689


### Philosopher Network

We'll now do the same for the philosopher network. However, the most important philosophers aren't the ones which **were influenced** by many and/or important philosophers but ones who **influenced** many and/or important other philosophers. So, we need to take a graph with the reverse of each edge to run our implementation of PageRank on.

We do this and run our two PageRank implementations:

In [6]:
import pandas as pd
import numpy as np

edgelist_df = pd.read_csv("data/philosophers/id_edgelist.txt", 
                       sep=",", header=None, names=["source id", "target id"])
edgelist = edgelist_df.to_numpy()
adj_matrix = np.zeros((edgelist.max(), edgelist.max()))
for e in edgelist:
    adj_matrix[e[0]-1][e[1]-1] = 1
    
d = 0.85
adj_matrix = make_stochastic(adj_matrix)
print("Running algebraic version of PageRank...")
results_alg = PageRank_algebraic(adj_matrix, d)
# print("Results: ")
# print(results_alg)
print("Running iterative version of PageRank...")
results_iter = PageRank_iterative(adj_matrix, d)
# print("Results: ")
# print(results_iter)
print("Correlation coefficient between results with the two methods: ")
print(np.corrcoef(results_alg.flatten(), results_iter.flatten())[0][1])

Running algebraic version of PageRank...
Running iterative version of PageRank...
Converged after  15  iterations
Correlation coefficient between results with the two methods: 
0.9999999519454222


Afterwards, we'll print the top nodes of our graph based in PageRank scores (as before, we'll use the results from the algebraic method):

In [7]:
nodes_df = pd.read_csv("data/philosophers/matchings.txt", 
                       sep=",", header=None, names=["philosopher", "id"])
nodes_df["PageRank"] = results_alg.flatten()
nodes_df.sort_values(by="PageRank", ascending=False, inplace=True)
nodes_df.head(10)

,philosopher,id,PageRank
665,aristotle,666,0.025997
310,plato,311,0.019387
986,immanuel_kant,987,0.014274
746,john_locke,747,0.009887
375,bertrand_russell,376,0.009865
1188,thomas_aquinas,1189,0.009309
989,cicero,990,0.008775
99,rené_descartes,100,0.008181
715,david_hume,716,0.008140
569,friedrich_nietzsche,570,0.007996


And finally we'll check the strengh of the relationship between PageRank and other Centrality metrics:

In [8]:
import networkx as nx

number_of_nodes = edgelist.max()
philosophers_graph = nx.DiGraph()
philosophers_graph.add_edges_from(edgelist)
flat_results = results_alg.flatten()
print("Correlation between PageRank and:")
# In degree
in_degree_dict = nx.in_degree_centrality(philosophers_graph)
in_degree_centrality = np.zeros(number_of_nodes)
for x in in_degree_dict:
    in_degree_centrality[x-1] = in_degree_dict[x]
print("In-degree Centrality : ", np.corrcoef(flat_results, in_degree_centrality)[0][1])
# Closeness Centrality
closeness_dict = nx.closeness_centrality(philosophers_graph)
closeness_centrality = np.zeros(number_of_nodes)
for x in closeness_dict:
    closeness_centrality[x-1] = closeness_dict[x]
print("Closeness Centrality : ", np.corrcoef(flat_results, closeness_centrality)[0][1])
# Betweenness Centrality
betweenness_dict = nx.betweenness_centrality(philosophers_graph)
betweenness_centrality = np.zeros(number_of_nodes)
for x in betweenness_dict:
    betweenness_centrality[x-1] = betweenness_dict[x]
print("Betweenness Centrality : ", np.corrcoef(flat_results, betweenness_centrality)[0][1])
# Eigenvector Centrality
eigenvector_dict = nx.eigenvector_centrality(philosophers_graph)
eigenvector_centrality = np.zeros(number_of_nodes)
for x in eigenvector_dict:
    eigenvector_centrality[x-1] = eigenvector_dict[x]
print("Eigenvector Centrality : ", np.corrcoef(flat_results, eigenvector_centrality)[0][1])

Correlation between PageRank and:
In-degree Centrality :  0.6593748879083564
Closeness Centrality :  0.24647552993932617
Betweenness Centrality :  0.8809129230695634
Eigenvector Centrality :  0.5483149457490674


## Comments on results & final thoughts
--------------------------------

We can see that even though PageRank doesn't value all links equally, it still has a very high correlation with the In-degree Centrality for both graphs used for our experiments. The Closeness Centrality has a lower Pearson's coefficient with PageRank but surely an important one. 

The coefficients between PageRank and Betweenness Centrality and between PageRank and Eigenvector Centrality are much stronger in the second graph. This is strange, but it may occur due to the second graph being denser or due to a certain particularity in the structure of one of the graphs.

As stated in the introduction, PageRank by itself can't get us very far when we try to extract the best search results for a term, most importantly because it doesn't take into account the term searched or the content of each webpage (node). However, it still is a very usefull tool for information retrieval from graph-like structures and understanding it may serve as a first step for someone wanting to dive deeper into information retrieval and into the way search engines work.

## Sources

* Wikipedia
* A survey of eigenvector methods for Web Information Retrieval (Amy N. Langville & Carl D. Meyer)
* PageRank Beyond the Web (David F. Gleich)